# Topic Modeling (EM algorithm and Gibbs Sampling)

## Probabilistic Latent Semantic Analysis with the EM Algorithm

In this homework you will exercise your expertise on the EM algorithm and Gibbs sampling to apply it to Probabilistic Latent Semantic Analysis (pLSA) and Latent Dirichlet Allocation.

### Part I: The pLSAModel

Recall that our data model for pLSA will consist of a set of documents $D$, and each document is modeled as a bag of words over dictionary $W$, we denote $x_{w,d}$ as the number of times word $w \in W$ appears in document $d \in D$.

#### Warmup: A simple multinomial model

Before we introduce the concept of topics, let's build a simple model based on frequency of word occurences to get used to Maximum Likelihood Estimation for multinomial distributions. Specifically, letting $n_d$ be the number of words in document $d$, then we model each document $d$ as $n_d$ draws from a Multinomial distribution with parameters $\theta_{1,d},\ldots,\theta_{W,d}$ with $\theta_{w,d}$ the probability of drawing word $w$ in document $d$. Note that $\theta_{w,d} \geq 0$ for all $w \in W$, and $\sum_w \theta_{w,d} = 1$.

With this model in place, the probability of observing the set of words in document $d$ is given by

$$
Pr(d|\theta_d) \varpropto \prod_{w=1}^{W} \theta_{w,d}^{x_{w,d}}
$$

where $\theta_d$ collects parameters $\{\theta_{1,d},\ldots,\theta_{W,d}\}$.

**Problem 1**: Prove that Maximum Likelihood Estimates (MLE) are given by 

$$\hat{\theta}_{w,d} = \frac{x_{w,d}}{n_d}$$, 

that is, the number of times word $w$ appears in document $d$ divided by the total number of words in document $d$.

_Hints_:

- Write MLE estimation problem as a _constrained_ maximization problem

- Write out the Lagrangian $L(\theta_d,\lambda, \nu)$ (see lecture slides) for this maximization problem.

- Use optimality conditions from lecture to solve

**Problem 1 Answer**

The log likelihood of the model is given by 

$$
\mathscr{L}(\theta_d) = \sum_{w=1}^W x_{w,d} \log \theta_{w,d}
$$

So, the MLE problem in standard form is

\begin{eqnarray}
\min_{\theta_d} & -\sum_{w=1}^W x_{w,d} \log \theta_{w,d} \\
\textrm{s.t.} & -\theta_{w,d} \leq 0 \; \forall w \\
{} & \sum_{w=1}^W \theta_{w,d} = 1
\end{eqnarray}

The Lagrangian of the problem is then

$$
L(\theta_d, \lambda, \nu) = -\sum_{w=1}^W x_{w,d} \log \theta_{w,d} + \sum_{w,d} \lambda_{w,d} (-\theta_{w,d}) + \nu(\sum_{w=1}^W \theta_{w,d} - 1)
$$

To solve, we use optimality conditions. First, the gradient of Lagrangian wrt to parameter should be zero:

\begin{eqnarray}
\frac{\partial L}{\partial \theta_{w,d}} = \frac{-x_{w,d}}{\theta_{w,d}} - \lambda_{w,d} + \nu \overset{\mathrm{set}}{=} 0 & \Rightarrow \\
\theta_{w,d} = \frac{- x_{w,d}}{\lambda_{w,d} - \nu} & {}
\end{eqnarray}

From complementarity and dual feasibility we can set $\lambda_{w,d}=0$.

From primal feasibility we have

\begin{eqnarray}
\sum_{w,d} \theta_{w,d} = 1 & \Rightarrow \\
\frac{\sum_{w,d} x_{w,d}}{\nu} = 1 & \Rightarrow \\
\nu = \sum_{w,d} x_{w,d} & \Rightarrow \\
\nu = n_d & \Rightarrow \\
\theta_{w,d} = \frac{x_{w,d}}{n_d} & {}
\end{eqnarray}

### A fully observed topic model

Let's introduce topics now. Instead of modeling each document as $d \sim \mathrm{Mult}(\{\theta_{1,d},\ldots,\theta_{W,d}\})$ over words, we model each document as a distribution over $T$ _topics_ as $d \sim \mathrm{Mult}(\{p_{1,d},\ldots,p_{T,d}\})$. In turn, each topic $t=1,\ldots,T$ is modeled as a distribution $t \sim \mathrm{Mult}(\{\theta_{1,t},\ldots,\theta_{W,t}\})$ over words. Note that the topics are shared across documents in dataset.

In pLSA, we learn topic distributions from observations by a soft assignment of each word occurence to topics using the EM algorithm. We will denote these _latent_ word-topic assignments as $\Delta_{w,d,t}$ to represent the number of times word $w$ was assigned to topic $t$ in document $d$.

Of course, we do not observe any of these latent word-topic assignments. However, it is helpful to think of the fully observed case to develop the EM algorithm. 

Assuming we observe word occurences $x_{w,d}$ and latent word-topic assignments $\Delta_{w,d,t}$ such that $\sum_t \Delta_{w,d,t} = x_{w,d}$ the full data probability is given by

$$
\mathrm{Pr}(D|\{p_d\},\{\theta_t\}) = \prod_{d=1}^D \prod_{w=1}^{W} \prod_{t=1}^T p_{t,d}^{\Delta_{w,d,t}}\theta_{w,t}^{\Delta_{w,d,t}}
$$



**Problem 2**: Prove that MLEs are given by

$$
\hat{p}_{t,d} = \frac{\sum_{w=1}^W \Delta_{w,d,t}}{\sum_{t=1}^T \sum_{w=1}^W \Delta_{w,d,t}}
$$

and

$$
\hat{\theta}_{w,t} = \frac{\sum_{d=1}^D \Delta_{w,d,t}}{\sum_{w=1}^W \sum_{d=1}^D \Delta_{w,d,t}}
$$

**Problem 2 Answer**

The log likelihood of the model is given by 

$$
\mathscr{L}(\theta_d) = \sum_{d=1}^D\sum_{w=1}^W \sum_{t=1}^T \Delta_{w,d,t} \log p_{t,d} + \Delta_{w,d,t} \log \theta_{w,t}
$$

So, the MLE problem in standard form is

\begin{eqnarray}
\min_{p_d,\theta_t} & -\sum_{d=1}^D\sum_{w=1}^W\sum_{t=1}^T \Delta_{w,d,t} p_{t,d} + \Delta_{w,d,t} \log \theta_{w,t} \\
\textrm{s.t.} & -p_{t,d} \leq 0\; \forall t,d \\
{} & -\theta_{w,t} \leq 0 \; \forall w,t \\
{} & \sum_{t=1}^T p_{t,d} = 1 \; \forall d \\
{} & \sum_{w=1}^W \theta_{w,t} = 1 \; \forall t
\end{eqnarray}

The Lagrangian of the problem is then

$$
L(p_{d},\theta_{t}, \lambda^p_d, \lambda^{\theta}_t, \nu^d, \nu^t) = -\sum_{d=1}^D\sum_{w=1}^W \sum_{t=1}^T \Delta_{w,d,t} \log p_{t,d} + \Delta_{w,d,t} \log \theta_{w,t} + \sum_{t,d} \lambda^p_{t,d} (-p_{t,d}) + \sum_{w,t} \lambda^{\theta}_{w,t} (-\theta_{w,t}) + \sum_{d=1}^D \nu^p_d(\sum_{t=1}^T p_{t,d}) + \sum_{t=1}^T \nu^{\theta}_t (\sum_{w=1}^W \theta_{w,t} - 1)
$$
To solve for $p_{t,d}$, we use optimality conditions. First, the gradient of Lagrangian wrt to parameter should be zero:

\begin{eqnarray}
\frac{\partial L}{\partial p{t,d}} = \frac{-\sum_{w=1}^W \Delta_{w,d,t}}{p_{t,d}} - \lambda^p_{t,d} + \nu^p \overset{\mathrm{set}}{=} 0 & \Rightarrow \\
p_{t,d} = \frac{- \sum_{w=1}^W \Delta_{w,d,t}}{\lambda^p_{t,d} - \nu^p_d} & {}
\end{eqnarray}

From complementarity and dual feasibility we can set $\lambda^p_{t,d}=0$.

From primal feasibility we have

\begin{eqnarray}
\sum_{t} p_{t,d} = 1 & \Rightarrow \\
\frac{\sum_{t} \sum_{w=1}^W \Delta_{w,d,t}}{\nu^p_d} = 1 & \Rightarrow \\
\nu^p_d = \sum_t \sum_{w} \Delta_{w,d,t} & \Rightarrow \\
p_{t,d} = \frac{\sum_w \Delta_{w,d,t}}{\sum_t \sum_w \Delta_{w,d,t}} & {}
\end{eqnarray}

To solve for $\theta_{w,t}$, we use optimality conditions. First, the gradient of Lagrangian wrt to parameter should be zero:

\begin{eqnarray}
\frac{\partial L}{\partial \theta_{w,t}} = \frac{- \sum_d \Delta_{w,d,t}}{\theta_{w,t}} - \lambda^{\theta}_{w,t} + \nu^{\theta}_t \overset{\mathrm{set}}{=} 0 & \Rightarrow \\
\theta_{w,t} = \frac{- \sum_d \Delta_{w,d,t}}{\lambda^{\theta}_{w,t} - \nu^{\theta}_t} & {}
\end{eqnarray}

From complementarity and dual feasibility we can set $\lambda^{\theta}_{w,t}=0$.

From primal feasibility we have

\begin{eqnarray}
\sum_{w} \theta_{w,t} = 1 & \Rightarrow \\
\frac{\sum_w \sum_{d} \Delta_{w,d,t}}{\nu^{\theta}_t} = 1 & \Rightarrow \\
\nu^{\theta}_t = \sum_w \sum_{d} \Delta_{w,d,t} & \Rightarrow \\
\theta_{w,t} = \frac{\sum_d \Delta_{w,d,t}}{\sum_w \sum_d \Delta_{w,d,t}} & {}
\end{eqnarray}


## Part II: pLSA with EM Algorithm

Denote the _responsibility_ of topic $t$ for the occurences of word $w$ in document $d$ as $\gamma_{w,d,t}=E[\Delta_{w,d,t}|\{p_d\},\{\theta_t\}]$

**Problem 3**: Write out the M-step for the EM algorithm based on the result of Problem 2


**Problem 3 Answer**

$$
\hat{p}_{t,d} = \frac{\sum_{w=1}^W \gamma_{w,d,t}}{\sum_{t=1}^T \sum_{w=1}^W \gamma_{w,d,t}}
$$

$$
\hat{\theta}_{w,t} = \frac{\sum_{d=1}^D \gamma_{w,d,t}}{\sum_{w=1}^W \sum_{d=1}^D \gamma_{w,d,t}}
$$

**Problem 4**: Show that the E-step for the EM algorithm, i.e., the update $\gamma_{d_j,t}$ given current set of parameters $\{p_d\}$ and $\{\theta_t\}$ is given by

$$
\gamma_{w,d,t} = x_{w,d} \times \frac{p_{t,d}\theta_{w,t}}{\sum_{t'=1}^T p_{t',d}\theta_{w,t'}}
$$

**Problem 4 Answer**

In the EM algorithm we set, and using Bayes Rule

\begin{eqnarray}
\gamma_{w,d,t} & = & E[\Delta_{w,d,t} | w=W, d=D; p_{t,d}, \theta_{w,t}] \\
{} & = & x_{w,d} p(T=t|W=w,D=d; p_{t,d}, \theta_{w,t}) \\
{} & = & x_{w,d} \frac{p(W=w|T=t;p_{t,d},\theta_{w,t})p(T=t|D=d;p_{t,d})}{p(W=w|D=d;p_{t,d}\theta_{w,t})} \\
{} & = & x_{w,d} \frac{p_{t,d}\theta_{w,t}}{\sum_{t'=1}^T p_{t',d}\theta_{w,t'}}
\end{eqnarray}



## Part III: Simulating data

**Problem 5** Complete the data simulation data in file `topic_lib/simulation.py` See lecture notes on how to do this.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from topic_lib.simulation import simulate_data

# set simulation parameters
num_docs = 20
num_words = 100
num_topics = 3
num_words_per_doc = 20

x, sim_delta, sim_p, sim_theta = simulate_data(num_words, num_docs, num_topics, num_words_per_doc)

In [3]:
# let's run a few assertions to check your implementation

# check the size of data matrix x
assert(x.shape == (num_words, num_docs))

# check that the total number of words in a document is correct
assert(np.all(np.sum(x, axis=0) == num_words_per_doc))

# check the size of simulated latent counts delta
assert(sim_delta.shape == (num_words, num_docs, num_topics))

# check that the sum of delta across topics equals the counts in data matrix x
assert(np.allclose(np.sum(sim_delta, axis=2), x))

# check the size of matrix p
assert(sim_p.shape == (num_topics, num_docs))

# check that p is normalized properly
assert(np.allclose(np.sum(sim_p, axis=0), np.ones((num_docs))))

# check the size of matrix theta
assert(sim_theta.shape == (num_words, num_topics))

# check that theta is normalized properly
assert(np.allclose(np.sum(sim_theta, axis=0), np.ones((num_topics))))

### Part IV: pLSA using EM

Implement pLSA using the EM updates from problems 3 and 4.

Notes:

- For the pLSA topic model we set out here, the probability of the observed word-document occurences is given by mixture distribution

$$
Pr(D|\{p_d\},\{\theta_t\}) = 
\prod_{d=1}^D \prod_{w=1}^W \left( \sum_{t=1}^T p_{t,d} \theta_{w,t} \right)^{x_{w,d}}
$$

- Complete the implementation in file `topic_lib/em.py`

- You will need to initialize parameters $\{p_d\}$ and $\{\theta_t\}$ (see lecture notes on the Dirichlet distribution)

- You will need to test for convergence

- You will need to deal with local minima (i.e, use multiple random initial points and choose the model that has largest likelihood).

- test your function on the small simulation dataset, i.e., from the data you generate above


In [4]:
from topic_lib.em import plsa_em

p, theta, llik = plsa_em(x, num_topics=3, verbose=False)

In [5]:
from topic_lib.simulation import compare_topics

# let's run a few assertions to check your implementation

# check the shape of the p estimate
assert(p.shape == (num_topics, num_docs))

# make sure the p estimate is properly normalized
assert(np.allclose(np.sum(p, axis=0), np.ones((num_docs))))
# let's see if you got close to the simulation p
# we check it to see if you identify the "important" topic in each document
# in the simulated p better than random chance
topic_assignment = compare_topics(sim_theta, theta)
print(topic_assignment)

match_rate = np.mean(np.argmax(p[topic_assignment,:],axis=0) == np.argmax(sim_p,axis=0))
print("Important topic in document match rate: {}".format(match_rate))
print("Important topic in document random match rate: {}\n".format(1. / num_topics))
assert(match_rate > 1. / num_topics)

# check the shape of the theta estimate
assert(theta.shape == (num_words, num_topics))

# make sure the theta estimate is properly normalized
assert(np.allclose(np.sum(theta, axis=0), np.ones((num_topics))))

# let's see if you get close to the simulation theta
# we check it to see if you identify the "important" words
# in each topic
match_rates = np.zeros((num_topics))
for t in range(num_topics):
    imp_words_sim = np.argsort(sim_theta[:,t])[-5:]
    imp_words = np.argsort(theta[:,topic_assignment[t]])[-5:]
    match_rates[t] = sum([w in imp_words_sim for w in imp_words]) / 5
    print("Important words in topic {} match rate: {}".format(t, match_rates[t]))

print("Important words in topic random match rate: {}\n".format(5. / num_words))
assert(np.all(match_rates > 5. / num_words))

# let's see how close you got to the log likelihood of the simulation parameters
from topic_lib.em import get_loglik
sim_ll = get_loglik(x, np.log(sim_p), np.log(sim_theta))
est_ll = get_loglik(x, np.log(p), np.log(theta))
print("Log likelihood for simulation parameters: {}".format(sim_ll))
print("Log likelhood of estimated parameters: {}".format(est_ll))

relative_dev = np.abs( sim_ll - est_ll ) / np.abs(sim_ll)
print("Relative absolute deviation: {}".format(relative_dev))
assert(relative_dev < 0.1)

[2 1 0]
Important topic in document match rate: 0.75
Important topic in document random match rate: 0.3333333333333333

Important words in topic 0 match rate: 1.0
Important words in topic 1 match rate: 0.6
Important words in topic 2 match rate: 0.6
Important words in topic random match rate: 0.05

Log likelihood for simulation parameters: -1625.8366488092925
Log likelhood of estimated parameters: -1488.3715689541239
Relative absolute deviation: 0.0845503636271475


## Part V: LDA with Gibbs Sampling

Implement Latent Dirichlet Annotation with Gibbs Sampling. See lecture notes for details.

Complete the implementation in file `topic_lib/gibbs.py`

In [9]:
from topic_lib.gibbs import lda_gibbs

p, theta = lda_gibbs(x, num_topics=3, verbose=False)

In [10]:
# let's run a few assertions to check your implementation

# check the shape of the p estimate
assert(p.shape == (num_topics, num_docs))

# make sure the p estimate is properly normalized
assert(np.allclose(np.sum(p, axis=0), np.ones((num_docs))))

# let's see if you got close to the simulation p
# we check it to see if you identify the "important" topic in each document
# in the simulated p better than random chance
topic_assignment = compare_topics(sim_theta, theta)

match_rate = np.mean(np.argmax(p[topic_assignment,:],axis=0) == np.argmax(sim_p,axis=0))
print("Important topic in document match rate: {}".format(match_rate))
print("Important topic in document random match rate: {}\n".format(1. / num_topics))
assert(match_rate > 1. / num_topics)

# check the shape of the theta estimate
assert(theta.shape == (num_words, num_topics))

# make sure the theta estimate is properly normalized
assert(np.allclose(np.sum(theta, axis=0), np.ones((num_topics))))

# let's see if you get close to the simulation theta
# we check it to see if you identify the "important" words
# in each topic
match_rates = np.zeros((num_topics))
for t in range(num_topics):
    imp_words_sim = np.argsort(sim_theta[:,t])[-5:]
    imp_words = np.argsort(theta[:,topic_assignment[t]])[-5:]
    match_rates[t] = sum([w in imp_words_sim for w in imp_words]) / 5
    print("Important words in topic {} match rate: {}".format(t, match_rates[t]))

print("Important words in topic random match rate: {}\n".format(5. / num_words))
assert(np.all(match_rates > 5. / num_words))

# let's see how close you got to the log likelihood of the simulation parameters
from topic_lib.em import get_loglik
sim_ll = get_loglik(x, np.log(sim_p), np.log(sim_theta))
est_ll = get_loglik(x, np.log(p), np.log(theta))
print("Log likelihood for simulation parameters: {}".format(sim_ll))
print("Log likelhood of estimated parameters: {}".format(est_ll))

relative_dev = np.abs( sim_ll - est_ll ) / np.abs(sim_ll)
print("Relative absolute deviation: {}".format(relative_dev))
assert(relative_dev < 0.1)

Important topic in document match rate: 0.85
Important topic in document random match rate: 0.3333333333333333

Important words in topic 0 match rate: 1.0
Important words in topic 1 match rate: 0.6
Important words in topic 2 match rate: 0.4
Important words in topic random match rate: 0.05

Log likelihood for simulation parameters: -1625.8366488092925
Log likelhood of estimated parameters: -1555.2428630679997
Relative absolute deviation: 0.043419974443922926


<ipython-input-10-fbf93db57ac8>:41: RuntimeWarning: divide by zero encountered in log
  est_ll = get_loglik(x, np.log(p), np.log(theta))


## Part IV: Applying Methods

Use your pLSA and LDA implementations to learn topics from the 20 newsgroups dataset. Utilities to
download and prepare the dataset is provided in file `topic_lib/newsgroups.py`. To run the
`get_docmat` function you will need to install packages gensim and nltk:

```
conda install -c anaconda gensim
conda install -c anaconda nltk
```

Compare topics learned from pLSA and LDA with number of topics $T=8$

To perform the comparison, print the top 5 words if each topic for each model (pLSA and LDA) (using function `get_topic_words` from file `topic_lib/newsgroups.py`.

(a) Do the topics they each return sensible?  
(b) Do the topics for one method make more sense than the other?


In [11]:
from topic_lib.newsgroups import get_docmat

newsgroups_mat = get_docmat()
print(newsgroups_mat.shape)

Found saved docmat file. Loading...
(1590, 1769)


In [14]:
# run each of the methods 
# (modify num_restarts and max_iter here)
em_p, em_theta, _ = plsa_em(newsgroups_mat, num_topics=8, verbose=True)

Run 0 of 3
It: 0, loglik: -717479.41727, old_q: -902172.52273, new_q: -864775.98201
It: 1, loglik: -714163.81412, old_q: -867626.72596, new_q: -865866.31294
It: 2, loglik: -711078.11246, old_q: -863412.11736, new_q: -861879.41889
It: 3, loglik: -707608.77947, old_q: -855862.25964, new_q: -854166.56771
It: 4, loglik: -703586.16845, old_q: -845339.50685, new_q: -843357.10803
It: 5, loglik: -699097.24887, old_q: -832448.67954, new_q: -830192.72407
It: 6, loglik: -694415.35528, old_q: -818171.37009, new_q: -815767.72407
It: 7, loglik: -689817.07790, old_q: -803724.08713, new_q: -801332.75704
It: 8, loglik: -685453.39601, old_q: -790057.53801, new_q: -787775.97591
It: 9, loglik: -681383.88885, old_q: -777595.64560, new_q: -775461.81932
It: 10, loglik: -677654.55154, old_q: -766385.79923, new_q: -764421.79584
It: 11, loglik: -674291.28610, old_q: -756392.24742, new_q: -754617.29374
It: 12, loglik: -671268.41821, old_q: -747561.92667, new_q: -745968.94124
It: 13, loglik: -668558.33424, old_q:

In [16]:
# modify num_rounds parameter here
gibbs_p, gibbs_theta = lda_gibbs(newsgroups_mat, num_topics=8, verbose=True)

Iteration 0, log_likelihood: -719062.0456314915
Iteration 10, log_likelihood: -696587.6542194713
Iteration 20, log_likelihood: -675002.659430214
Iteration 30, log_likelihood: -670130.9126155886
Iteration 40, log_likelihood: -668402.0192358766
Iteration 50, log_likelihood: -667357.0695054223
Iteration 60, log_likelihood: -667036.1813403091
Iteration 70, log_likelihood: -666909.2768358274
Iteration 80, log_likelihood: -666824.9369911373
Iteration 90, log_likelihood: -666764.8225244443
Iteration 100, log_likelihood: -666754.6135151176
Iteration 110, log_likelihood: -666638.3832313546
Iteration 120, log_likelihood: -666821.3457409298
Iteration 130, log_likelihood: -666717.355475865
Iteration 140, log_likelihood: -666552.8770153266
Iteration 150, log_likelihood: -666553.0518274568
Iteration 160, log_likelihood: -666682.4288232364
Iteration 170, log_likelihood: -666355.8014198397
Iteration 180, log_likelihood: -666390.5686715703
Iteration 190, log_likelihood: -666093.8238488914


In [17]:
from topic_lib.newsgroups import print_important_words

# print important words from EM estimate
print_important_words(em_theta)

Dictionary(1590 unique tokens: ['acceler', 'adapt', 'answer', 'base', 'brave']...)
['lose', 'cub', 'brave', 'score', 'pitch']
['netcom', 'major', 'alomar', 'roger', 'player']
['power', 'austin', 'nasa', 'uiuc', 'david']
['indiana', 'type', 'manag', 'sound', 'keyboard']
['machin', 'simm', 'monitor', 'card', 'drive']
['medic', 'pain', 'diseas', 'patient', 'doctor']
['dyer', 'state', 'research', 'health', 'studi']
['pittsburgh', 'food', 'gordon', 'bank', 'pitt']


In [18]:
# print important words from Gibbs estimates
print_important_words(gibbs_theta)

Dictionary(1590 unique tokens: ['acceler', 'adapt', 'answer', 'base', 'brave']...)
['center', 'medic', 'state', 'research', 'health']
['univ', 'pittsburgh', 'gordon', 'bank', 'pitt']
['list', 'berkeley', 'water', 'type', 'keyboard']
['scsi', 'simm', 'monitor', 'card', 'drive']
['major', 'roger', 'averag', 'david', 'player']
['experi', 'sensit', 'nasa', 'dyer', 'food']
['cub', 'season', 'lose', 'score', 'pitch']
['treatment', 'diseas', 'caus', 'patient', 'doctor']


**Answer questions (a) and (b) here**

(a) Both methods return for the most part sensible topics related to medicine, sports and computer hardware. Some topics in both cases include individual words that may not be relevant to that topic (e.g, topic 4 for em). 

(b) Topics from both methods are very similar.